# Extract Noise Embedding

## Get the Longest Common Subsequence

In [1]:
def LCS(seq1, seq2):
    """
    Initialise a DP array of len(seq2)+1 columns and len(seq1)+1 rows
    the extra column and row is to denote the empty sequence as a base case
    """
    dp = [[0 for j in range(len(seq2)+1)] for i in range(len(seq1)+1)]

    # Starting from the bottom right most cell and moving from right to left, start the bottom up approach
    for i in range(len(seq1)-1,-1,-1):
        for j in range(len(seq2)-1,-1,-1):
            # If the elements of seq1 and seq2 match, store a 1 + value at the diagonal cell
            # Store 1 because the elements match
            # Get value from diagonal cell because both elements match so our subproblem moves (i+1,j+1)
            if seq1[i]==seq2[j]:
                dp[i][j] = 1 + dp[i+1][j+1]
            # If the elements of seq1 and seq2 do not match, get the value from its right or bottom cell, taking the max
            # We do this to get the max longest common sub sequence of our sub problems after moving (i+1,j) or (i,j+1)
            else:
                dp[i][j] = max(dp[i][j+1], dp[i+1][j])
    # The very first element stores the LCS for the 2 full sequences, building up its value in the bottom up approach
    return dp[0][0]

In [2]:
LCS('abcde','ace')

3

In [3]:
def LCS_return_seq(seq1, seq2):
    """
    Initialise a DP array of len(seq2)+1 columns and len(seq1)+1 rows
    the extra column and row is to denote the empty sequence as a base case
    """
    dp = [[0 for j in range(len(seq2)+1)] for i in range(len(seq1)+1)]
    # Fill up the DP array where each cell contains the LCS of the subproblems
    # Starting from the bottom right most cell and moving from right to left, start the bottom up approach
    for i in range(len(seq1)-1,-1,-1):
        for j in range(len(seq2)-1,-1,-1):
            # If the elements of seq1 and seq2 match, store a 1 + value at the diagonal cell
            # Store 1 because the elements match
            # Get value from diagonal cell because both elements match so our subproblem moves (i+1,j+1)
            if seq1[i]==seq2[j]:
                dp[i][j] = 1 + dp[i+1][j+1]
            # If the elements of seq1 and seq2 do not match, get the value from its right or bottom cell, taking the max
            # We do this to get the max longest common sub sequence of our sub problems after moving (i+1,j) or (i,j+1)
            else:
                dp[i][j] = max(dp[i][j+1], dp[i+1][j])
    
    # Get the actual subsequence
    # Re-initialise the pointers
    i = 0
    j = 0
    lcs = []

    while i < len(seq1) and j < len(seq2):
        # If the characters match at those positions, add the character
        if seq1[i]==seq2[j]:
            lcs.append(seq1[i])
            # Move diagonally as our subproblem now becomes i+1,j+1
            i+=1
            j+=1
        # If the characters don't match at that cell, we try going to the cell
        # with the greater value (either the right or down cell which are our subproblems)
        # We go to the cell with the greater value because a match was found on or near that cell
        elif dp[i+1][j]>=dp[i][j+1]:
            i+=1
        else:
            j+=1

    return lcs

In [4]:
LCS_return_seq('abcde','ace')

['a', 'c', 'e']

## Pad 2 sequences

In [5]:
def force_align(seq1,seq2):
    # Get the lcs between the 2 sequences
    lcs = LCS_return_seq(seq1, seq2)

    seq1_aligned = []
    seq2_aligned = []

    i = 0
    j = 0

    padding = "Pad"
    next_x = False

    """
    Big Idea: 
    - align the lcs tokens
    - for the out-of-lcs token in one sequence, align it with a padding token in the other sequence 
      to denote token level noise
    """
    for x in lcs:
        next_x = False
        while not next_x:

            # Case 1: seq1[i]==seq2[j]==x
            # Action
            # - append seq[i] to seq1_aligned
            # - append seq[j] to seq2_aligned
            # - i+1
            # - j+1
            if seq1[i]==x and seq2[j]==x:
                seq1_aligned.append(seq1[i])
                seq2_aligned.append(seq2[j])
                i+=1
                j+=1
                # Move to the next x in lcs
                next_x = True

            # Case 2: seq1[i]==x but seq[2]!=x
            # Action
            # - append padding to seq1_aligned to match the out-of-lcs token from seq2
            # - append out-of-lcs token seq2[j] to seq2_aligned
            # - j+1 to simulate that the j-th token has been matched by the padding token in seq1
            elif seq1[i]==x and seq2[j]!=x:
                seq1_aligned.append(padding)
                seq2_aligned.append(seq2[j])
                j+=1

            # Case 3 and 4: seq1[i]!=x but seq[2]==x as well as seq1[i]!=x but seq[2]!=x
            # Action
            # - append out-of-lcs token seq1[i] to seq1_aligned
            # - append padding to seq2_aligned to match the out-of-lcs token from seq1
            # - i+1 to simulate that the i-th token has been matched by the padding token in seq2
            # For the case where both don't match, we use the same logic. It'll result
            # in matched padding to out-of-lcs tokens in both seqs
            else:
                seq1_aligned.append(seq1[i])
                seq2_aligned.append(padding)
                i+=1

    # Once all the lcs tokens have been aligned
    # for the out-of-lcs token in one sequence, align it with a padding token in the other sequence
    while i < len(seq1):
        seq1_aligned.append(seq1[i])
        seq2_aligned.append(padding)
        i+=1

    while j < len(seq2):
        seq1_aligned.append(padding)
        seq2_aligned.append(seq2[j])
        j+=1

    return seq1_aligned, seq2_aligned


In [6]:
# Case where one col doesn't match
seq1 = ['a','j','c','d','e']
seq2 = ['a','z','b','c','e']
seq1_aligned, seq2_aligned = force_align(seq1,seq2)
print(seq1_aligned)
print(seq2_aligned)

['a', 'j', 'Pad', 'Pad', 'c', 'd', 'e']
['a', 'Pad', 'z', 'b', 'c', 'Pad', 'e']


**Given example**

Input:

```
1. a b c d e
2. a b c e
3. a b c d e
4. a b c e
5. a z b c e
```

Output:


```
1. a pad b c d   e
2. a pad b c pad e
3. a pad b c d   e
4. a pad b c pad e
5. a z   b c pad e
```

In [7]:
# 1 and 2
seq1 = ['a','b','c','d','e']
seq2 = ['a','b','c','e']
seq1_aligned, seq2_aligned = force_align(seq1,seq2)
print(seq1_aligned)
print(seq2_aligned)

['a', 'b', 'c', 'd', 'e']
['a', 'b', 'c', 'Pad', 'e']


In [8]:
# 1 and random case
seq1 = ['a','b','c','d','e']
seq2 = ['a','z','b','g','c','e']
seq1_aligned, seq2_aligned = force_align(seq1,seq2)
print(seq1_aligned)
print(seq2_aligned)

['a', 'Pad', 'b', 'Pad', 'c', 'd', 'e']
['a', 'z', 'b', 'g', 'c', 'Pad', 'e']


In [9]:
# 1 and 5
seq1 = ['a','b','c','d','e']
seq2 = ['a','z','b','c','e']
seq1_aligned, seq2_aligned = force_align(seq1,seq2)
print(seq1_aligned)
print(seq2_aligned)

['a', 'Pad', 'b', 'c', 'd', 'e']
['a', 'z', 'b', 'c', 'Pad', 'e']


In [10]:
# 1 (after 5) and 2
seq1 = ['a', 'Pad', 'b', 'c', 'd', 'e']
seq2 = ['a','b','c','e']
seq1_aligned, seq2_aligned = force_align(seq1,seq2)
print(seq1_aligned)
print(seq2_aligned)

['a', 'Pad', 'b', 'c', 'd', 'e']
['a', 'Pad', 'b', 'c', 'Pad', 'e']


## How to align N hypotheses

**What if we have a case where we have 2 different LCS**

In [11]:
seq1 = ['a','q','c','r','e']
seq2 = ['a','s','c','t','e']
seq3 = ['a','u','g','v','p','b','e']
seq4 = ['a','z','w','g','p','x','e']

In [12]:
seq1_aligned, seq2_aligned = force_align(seq1,seq2)
print(seq1_aligned)
print(seq2_aligned)

['a', 'q', 'Pad', 'c', 'r', 'Pad', 'e']
['a', 'Pad', 's', 'c', 'Pad', 't', 'e']


In [13]:
seq1_aligned, seq2_aligned = force_align(seq1,seq3)
print(seq1_aligned)
print(seq2_aligned)

['a', 'q', 'c', 'r', 'Pad', 'Pad', 'Pad', 'Pad', 'Pad', 'e']
['a', 'Pad', 'Pad', 'Pad', 'u', 'g', 'v', 'p', 'b', 'e']


In [14]:
seq1_aligned, seq2_aligned = force_align(seq3,seq4)
print(seq1_aligned)
print(seq2_aligned)

['a', 'u', 'Pad', 'Pad', 'g', 'v', 'p', 'b', 'Pad', 'e']
['a', 'Pad', 'z', 'w', 'g', 'Pad', 'p', 'Pad', 'x', 'e']
